In [ ]:
import geopandas
import rioxarray
import shapely
import pdal
import numpy
import json
import pathlib
import pdal
import time

# LiDAR tile list

In [ ]:
lidar_file_paths = list(pathlib.Path('C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\local_cache\\Wellington_2013').glob('*.laz'))

In [ ]:
crs = 2193
resolution = 10
catchment = geopandas.read_file(r'C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\waikanae_catchment.zip')
catchment = catchment.to_crs(crs)

In [ ]:
raster_size = [int((catchment.loc[0].geometry.bounds[2] - catchment.loc[0].geometry.bounds[0]) / resolution), 
               int((catchment.loc[0].geometry.bounds[3] - catchment.loc[0].geometry.bounds[1]) / resolution)]
raster_origin = [catchment.loc[0].geometry.bounds[0], catchment.loc[0].geometry.bounds[1]]

In [ ]:
extents = []
points = []
for i in range(2):
    pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(lidar_file_paths[i])},
                                  {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)},
                                  {"type":"filters.crop", "polygon":str(catchment.loc[0].geometry)}, 
                                  {"type":"filters.hexbin"}]
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()
    metadata=json.loads(pdal_pipeline.get_metadata())
    extents.append(shapely.wkt.loads(metadata['metadata']['filters.hexbin']['boundary']))
    points.append(pdal_pipeline.arrays)

In [ ]:
extents

In [ ]:
i = 0

In [ ]:
tmp_raster = "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\local_cache\\temp_1.tif"
dem_points = numpy.zeros_like(points[0], shape=[2])
radius =  resolution * numpy.sqrt(2); window_size = 0; idw_power = 2
pdal_pipeline_instructions = [
            {"type":  "writers.gdal", "resolution": resolution, "gdalopts": "a_srs=EPSG:" + str(crs), "output_type":["idw"], 
             "filename": tmp_raster, 
             "window_size": window_size, "power": idw_power, "radius": radius, 
             "origin_x": raster_origin[0], "origin_y": raster_origin[1], 
             "width": raster_size[0], "height": raster_size[1]}
        ]

start = time.time()
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [dem_points])
pdal_pipeline.execute();
end = time.time()
print(end - start)

In [ ]:
with rioxarray.rioxarray.open_rasterio(tmp_raster, masked=True) as dem:
    dem.plot()

In [ ]:
dem

In [ ]:
dem = rioxarray.rioxarray.open_rasterio(tmp_raster, masked=True)
dem.plot()

In [ ]:
dem.close()

In [ ]:
dem.plot()

In [ ]:
[dem.x.data.min(), dem.y.data.min()]

In [ ]:
tmp_raster_ = "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\local_cache\\temp_1_.tif"
dem = rioxarray.rioxarray.open_rasterio(tmp_raster_, masked=True)
dem.plot()

In [ ]:
pdal_pipeline_instructions[0]["origin_x"]

In [ ]:
pdal_pipeline_instructions = [
            {"type":  "writers.gdal", "resolution": resolution, "gdalopts": "a_srs=EPSG:" + str(crs), "output_type":["idw"], 
             "filename": tmp_raster_, 
             "window_size": window_size, "power": idw_power, "radius": radius, 
             "origin_x": extents[i].bounds[0]-resolution, "origin_y": extents[i].bounds[1]-resolution, 
             "width": int((extents[i].bounds[2]-extents[i].bounds[0]+2*resolution)/resolution), 
             "height": int((extents[i].bounds[3]-extents[i].bounds[1]+2*resolution)/resolution)}
        ]

start = time.time()
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [[]])
pdal_pipeline.execute();
end = time.time()
print(end - start)

In [ ]:
extents[0].bounds

In [ ]:
[dem.x.data.min(), dem.y.data.min()]

In [ ]:
raster_origin

In [ ]:
[dem.x.data.max(), dem.y.data.max()]

In [ ]:
raster_origin + raster_size